In [ ]:
import pandas as pd

In [ ]:
#Checking if GPU is running or not
!pip install transformers datasets sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.5 MB/s eta 0:00:00


In [ ]:
!pip install datasets

In [ ]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM

In [ ]:
model_checkpoint = "sayanmandal/t5-small_6_3-hi_en-to-en"

In [ ]:
raw_datasets=load_dataset('csv',data_files="Hinglish_Top.csv")

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['en_query', 'cs_query'],
        num_rows: 392428
    })
})

In [ ]:
all = raw_datasets["train"].shuffle(seed=42)

In [ ]:
all

Dataset({
    features: ['en_query', 'cs_query'],
    num_rows: 392428
})

In [ ]:
from datasets import Dataset

all_data = {
    'en_query': all['en_query'],
    'cs_query': all['cs_query'],
}

dataset_all = Dataset.from_dict(all_data)

In [ ]:
from sklearn.model_selection import train_test_split
train_data, t_v = train_test_split(dataset_all, test_size=0.3, random_state=42)



In [ ]:
train_data=pd.DataFrame(train_data)
train_data

,en_query,cs_query
0,Instead of my reminder going off on friday can...,mere reminder ko friday ko band hone ke bajaye...
1,how much of a delay for my trip home today,Aaj mere ghar ke trip par kitna delay hoga
2,Remind me about Alexa ' s party Friday at 2 pm .,Mujhe Friday 2 pm ko Alexa ki party ke baare m...
3,How much time does it take to drive from Pullm...,Pullman se Bellingham tak drive karne mei kitn...
4,Loving how Pandora introducing gosepel songs f...,Pandora ka use karke gospel songs ko unke styl...
...,...,...
274694,How many miles from America to Russia,America se Russia tak kitne miles hai
274695,Forecast of Brisbane for this weekend,Is weekend ke liye Brisbane ka forecast
274696,Please let me know if I will need an umbrella ...,Please mujhe bataye ke aaj mujhe umbrella ki z...
274697,Remind me to sign permission slip,mujhe permission slip sign karne ke liye yaad ...


In [ ]:
t_v

{'en_query': ['Message Jared and let him know that I will meet him at the comic book store at 3 pm .',
  'Create a country play list',
  'How long will it take to get home from here ?',
  'Remind me to make a dentists appointment in 6 months',
  'Delete the reminder to get my tires serviced .',
  'Will it be raining tomorrow ?',
  'Is it cloudy over there ?',
  'Could you give me the rain forecast in inches thanks',
  'Change the alarm sound to vibrate',
  'How many reminders to I have today ?',
  'Tell me how much it will rain in cm',
  'Set a reminder to turn windows updates off on the laptops for the upcoming test .',
  'Edit alarms',
  'how is the traffic in nola',
  'review reminders for next month to do with business',
  'Play my favorite playlist',
  'What time will I get to Tucson if I leave now with the current traffic',
  'Play bands like NoFX .',
  "what ' s my estimated drive between baltimore and DC",
  'What reminders do I have set for dinner plans this month ?',
  'set a

In [ ]:
t_v_2=pd.DataFrame(t_v)

In [ ]:
test_data,valid_data = train_test_split(t_v_2, test_size=0.3, random_state=42)

In [ ]:
test_data

,en_query,cs_query
30892,Best route to avoid traffic going to the Camde...,Camden aquarium jaate samay traffic se bachne ...
61571,Set an alarm for 6 am tomorrow,kal subah 6 baje ke liye alarm set karo
5688,How long is it supposed to snow for ?,kya ye snow kab tak padne wali hai ?
8365,Get me my family chat messages,Muje mere family chat messages bheje
26215,i need that alarm off now,mujhe abhi wo alarm band karna chahiye
...,...,...
76820,What ' s the 3 - day forecast ?,3 - day forecast kya hai
110268,send message to john to meet me at my house on...,mere birthday par muje meri ghar par milne ke ...
103694,what are my reminders for this week,is week ke liye mere reminders kya hai
860,Is NYC the city with the worst traffic,kya NYC sabse kharab traffic wala city hai


In [ ]:
print("train_data",len(test_data))
print("valid_data",len(valid_data))
print("train_data",len(train_data))


train_data 82410
valid_data 35319
train_data 274699


In [ ]:
test_data = Dataset.from_pandas(test_data)
valid_data = Dataset.from_pandas(valid_data)
train_data = Dataset.from_pandas(train_data)


In [ ]:
split = {
    'train_data_forMerge': train_data,
    'test_data_forMerge': test_data,
    'valid_data_forMerge': valid_data,
}
split

{'train_data_forMerge': Dataset({
     features: ['en_query', 'cs_query'],
     num_rows: 274699
 }),
 'test_data_forMerge': Dataset({
     features: ['en_query', 'cs_query', '__index_level_0__'],
     num_rows: 82410
 }),
 'valid_data_forMerge': Dataset({
     features: ['en_query', 'cs_query', '__index_level_0__'],
     num_rows: 35319
 })}

In [ ]:
from datasets import DatasetDict
split = DatasetDict(split)


In [ ]:
split

DatasetDict({
    train_data_forMerge: Dataset({
        features: ['en_query', 'cs_query'],
        num_rows: 274699
    })
    test_data_forMerge: Dataset({
        features: ['en_query', 'cs_query', '__index_level_0__'],
        num_rows: 82410
    })
    valid_data_forMerge: Dataset({
        features: ['en_query', 'cs_query', '__index_level_0__'],
        num_rows: 35319
    })
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:

def tokenizer_fn(batch):
  inputs = [ x  for x in batch["cs_query"]]
  targets = [x  for x in batch["en_query"]]

  tokenized_inputs = tokenizer(
    inputs, max_length=128, truncation=True)

  tokenized_targets = tokenizer(
    text_target=targets,  max_length=128,truncation=True)

  tokenized_inputs['labels'] = tokenized_targets['input_ids']
  return tokenized_inputs

In [ ]:
tokenized_datasets = split.map(
    tokenizer_fn,
    batched=True,
    remove_columns=split["train_data_forMerge"].column_names,
)


Map:   0%|          | 0/274699 [00:00<?, ? examples/s]

Map:   0%|          | 0/82410 [00:00<?, ? examples/s]

Map:   0%|          | 0/35319 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
pip install transformers[torch] -U


In [ ]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
  "finetuned-model",
  evaluation_strategy="no",
  #save_strategy="epoch",
  learning_rate=2e-5,
  per_device_train_batch_size=32,
  per_device_eval_batch_size=64,
  weight_decay=0.01,
  save_total_limit=3,
  num_train_epochs=3,
  predict_with_generate=True,
  fp16=True,
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train_data_forMerge"],
    eval_dataset=tokenized_datasets["test_data_forMerge"],
    data_collator=data_collator,
    tokenizer=tokenizer,

)

In [ ]:
trainer.evaluate()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 1.7339091300964355,
 'eval_runtime': 54.1079,
 'eval_samples_per_second': 1523.068,
 'eval_steps_per_second': 23.804}

In [ ]:
trainer.train()

Step,Training Loss
500,2.009900
1000,1.795100
1500,1.660200
2000,1.589400
2500,1.520900
3000,1.454300
3500,1.410100
4000,1.370100
4500,1.329800
5000,1.299000


TrainOutput(global_step=25755, training_loss=1.1547319409639436, metrics={'train_runtime': 2252.4172, 'train_samples_per_second': 365.872, 'train_steps_per_second': 11.434, 'total_flos': 7181980934231040.0, 'train_loss': 1.1547319409639436, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.7894511222839355,
 'eval_runtime': 51.5927,
 'eval_samples_per_second': 1597.32,
 'eval_steps_per_second': 24.965,
 'epoch': 3.0}

In [ ]:
trainer.save_model("my_saved_model")

In [ ]:
from transformers import pipeline
translator = pipeline("translation", model='my_saved_model', device=0)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/__init__.py:1049: UserWarning: "translation" task was used, instead of "translation_XX_to_YY", defaulting to "translation_en_to_de"
  warnings.warn(


In [ ]:
translator("Aaj mera day boht awfull tha")

[{'translation_text': 'Today my day was awful'}]

In [ ]:
#storing model directory as zip
import shutil

shutil.make_archive('output', 'zip', "/content/my_saved_model")


'/content/output.zip'

In [ ]:
#unzipping the model
import zipfile
import os

zip_file_path = '/content/output.zip'

output_directory = 'my_saved_model_2'

os.makedirs(output_directory, exist_ok=True)

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(output_directory)



In [ ]:
#testing the loaded model
from transformers import pipeline
translator2 = pipeline("translation", model='my_saved_model_2', device=0)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/__init__.py:1049: UserWarning: "translation" task was used, instead of "translation_XX_to_YY", defaulting to "translation_en_to_de"
  warnings.warn(


In [ ]:
translator2("Aap kaise ho")

[{'translation_text': 'How are you'}]